In [ ]:
# Import necessary libraries
import glob
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import pandas as pd
import verde as vd
import multiprocessing
from itertools import accumulate

from tqdm import tqdm

import xarray as xr

import time, array, random

from deap import base, creator, tools, algorithms

from CODES.modeling import create_seismic_model,calculate_parameters,calculate_parameters_from_vs
from CODES.dispersion_curves import create_velocity_model_from_profile,create_velocity_model_from_profile_vs,estimate_disp_from_velocity_model

# Text based on:

- Gallagher, K., & Sambridge, M. (1994). **Genetic algorithms: a powerful tool for large-scale nonlinear optimization problems**. Comput. Geosci., 20(7–8), 1229–1236.
- Fortin, F. A., Rainville, F. M., Gardner, M., Parizeau, M., and Gagné, C. **DEAP: Evolutionary Algorithms Made Easy**, Journal of Machine Learning Research, pp. 2171-2175, no 13, jul 2012.
- Haskell, N. A. ,1953, **The dispersion of surface waves on multi-layered media**, Bull. Seism. Soc. Am. 43, 17-34.
- Xia, J., Miller, R. D.,  and Park,C. B., 1999, **Estimation of near‐surface shear‐wave velocity by inversion of Rayleigh waves**, GEOPHYSICS 1999 64:3, 691-700.
- Yamanaka, H., and H. Ishida, 1996, **Application of genetic algorithms to an inversion of surface-wave dispersion data**, Bulletin of the Seismological Society of America, 86, 436–444.

# Inversion of Surface-Wave Dispersion Data

**Rayleigh-wave phase velocity is the function of four parameters (Xia et al, 1999):**
    - S-wave velocity;
    - P-wave velocity;
    - Density; and
    - Layer thickness.

In the process of retrieving a 2D Vs profile from a 3D layered earth model, the primary step involves extracting 1D Vs profiles at multiple locations across the study area. Since surface wave analysis is most sensitive to vertical variations in Vs, the lateral variations are effectively averaged out during data processing. This results in a 1D Vs model, often referred to as a layered earth model, where Vs is determined as a function of depth, while other parameters such as Vp and density (ρ) may be either included or held constant. Each 1D Vs profile is assigned to the central location of its respective receiver spread, representing the most characteristic subsurface structure at that point. By accumulating multiple such profiles, each tagged with a unique surface coordinate, a 2D Vs profile is constructed through spatial interpolation. This method enables a continuous Vs representation along a given transect while maintaining the fundamental assumption of a layered earth model.


# Inputs and outputs

In [ ]:
folder_model_name = 'MCP2_2'

### Directory for saving model and other related stuffs: 

In [ ]:
data_path = 'OUTPUT/'+folder_model_name+'/DATA/'

### Directory for saving figures: 

In [ ]:
figures_path = 'OUTPUT/'+folder_model_name+'/FIGURES/'

---------

# **Comparison and final analysis:**

In [ ]:
lst_receptor = np.arange(1, 50-1,5)*2

# Observed data

In [ ]:
obs_data = pd.read_feather(data_path+'observed_data.feather')
obs_data

In [ ]:
obs_depth_result_lst = []
obs_velocity_result_lst = []
obs_receptor_result_lst = []

depth_interval = -0.05

for dativ in obs_data.iterrows():

    depths = dativ[1]['depth'].tolist()[::-1]
    vels_mean = dativ[1]['velocity_s'].tolist()[::-1]

    depths_fine = []
    vels_fine = []

    # Iterate through depth intervals
    for j in range(len(depths) - 1):
        # Create a new depth array within the interval with finer sampling
        depths_interval = np.arange(depths[j], depths[j + 1], depth_interval)

        depths_fine.extend(depths_interval)
        # Repeat the velocity value within the interval
        vels_fine.extend([vels_mean[j]] * len(depths_interval))

    obs_receptor_result_lst.append([lst_receptor[dativ[0]]]*len(vels_fine))
    obs_depth_result_lst.append(depths_fine)
    obs_velocity_result_lst.append(vels_fine)
    
obs_receptor_result_lst = np.array([item for sublist in obs_receptor_result_lst for item in sublist])
obs_depth_result_lst = np.array([item for sublist in obs_depth_result_lst for item in sublist])
obs_velocity_result_lst = np.array([item for sublist in obs_velocity_result_lst for item in sublist])

# We'll test this on the air temperature data from Texas
obs_coordinates = (obs_receptor_result_lst,obs_depth_result_lst)
obs_velocitys = obs_velocity_result_lst
obs_region = (0,100,-2,0)

obs_shape = (100,100)

# Now we can set up a gridder for the decimated data
grd_obs = vd.KNeighbors().fit(obs_coordinates, obs_velocitys)

# The 'grid' method can still make a geographic grid if we pass in a projection
# function that converts lon, lat into the easting, northing coordinates that
# we used in 'fit'. This can be any function that takes lon, lat and returns x,
# y. In our case, it'll be the 'projection' variable that we created above.
# We'll also set the names of the grid dimensions and the name the data
# variable in our grid (the default would be 'scalars', which isn't very
# informative).
observed_grid = grd_obs.grid(
    region=obs_region,
    shape=obs_shape,
    dims=["depth","receptor"],
    data_names="velocity",
)

In [ ]:
fig,ax = plt.subplots(1, 1, figsize=(10, 5))

im = plt.imshow(observed_grid.velocity.data, cmap='cividis',extent=[0, 100, -2.0, 0],vmin=100,vmax=1000,aspect=25,origin='lower')

for idx,ix in enumerate(lst_receptor):
    ax.vlines(x=ix,ymin=-2.0,ymax=0,colors='k',lw=1.0,ls='--',alpha=0.7)
    ax.scatter(x=ix,y=0.05,color='k',marker='v')

ax.hlines(y=0,xmin=0,xmax=100,colors='k',lw=2,ls='-',alpha=1)
ax.set_xlabel('Distance (m)')
ax.grid(which='major', color='gray', linestyle='-', linewidth=0.5)
ax.grid(which='minor', color='gray', linestyle=':', linewidth=0.5, alpha=0.5)
ax.set_ylabel('Depth (m)')
plt.colorbar(im, ax=ax,fraction=0.15, shrink=0.5,label='Shear-wave velocity (m/s)')
fig.savefig(figures_path+'observed_slice_and_receptors.png')

# Inverted data

In [ ]:
df_inversion = pd.read_feather(data_path+'inversion_.feather')
df_inversion

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
for dativ in df_inversion.iterrows():
    ax.semilogy(dativ[1]['ngen'], dativ[1]['misfit'], '-')  # Sem o label
        
ax.set_xlabel('Number of generations')
ax.set_ylabel('Misfit value')
ax.axvline(x=len(dativ[1]['ngen'])-1, color='red', linestyle='--', label='Last gen')
ax.legend(loc='upper right')
ax.grid(True, which='both')
plt.tight_layout()
ax.tick_params(axis='both', which='both', direction='in', 
               top=True, bottom=True, left=True, right=True,
               labelbottom=True, labeltop=False, labelleft=True,
               labelright=True)
ax.grid(which='major', color='gray', linestyle='-', linewidth=0.5)
ax.grid(which='minor', color='gray', linestyle=':', linewidth=0.5, alpha=0.5)
ax.set_title('Misfit evolution')
      
fig.savefig(figures_path+'misfit_total_.png', dpi=300)

# The result was based on the three solutions with the lowest misfit values - Predicted data

In [ ]:
depth_result_lst = []
velocity_result_lst = [] 
receptor_result_lst = []

fig, axs = plt.subplots(1, 10, layout="constrained",figsize=(20, 5), facecolor='white',sharey=True)

for prof_n in df_inversion['profile'].unique():
    
    ax = axs[prof_n-1]

    # --------------
    # Observed data

    observed_profile_df = obs_data.loc[prof_n-1]
    ax.step(observed_profile_df['velocity_s'],observed_profile_df['depth'], color='gray',ls='-',lw=4,where='post',zorder=-1)

    # --------------
    # Predicted data
    
    profile_df = df_inversion[df_inversion['profile'] == prof_n]
    for dativ in profile_df.iterrows():
        ax.step(np.insert(dativ[1]['Vs'],0,dativ[1]['Vs'][0]),np.insert(np.cumsum(dativ[1]['thick']),0,0)*-1, color='k',alpha=0.8,lw=0.1,ls='-',where='post')

    sort_df = profile_df.sort_values(by='misfit_min').head(5)
    vs_std = sort_df["Vs"].apply(pd.Series).std().tolist()
    vs_mean = sort_df["Vs"].apply(pd.Series).mean().tolist()
    thick_mean = sort_df["thick"].apply(pd.Series).mean().tolist()
    misfit_min_mean = sort_df["misfit_min"].mean()

    # --------------------------------------------
    # Creating the grid at a specif depth interval

    depth_interval = -0.01

    depths = [0]+[-j for j in list(accumulate(thick_mean))]
            
    depths_fine = []
    vels_fine = []
          
    # Iterate through depth intervals
    for j in range(len(depths) - 1):
        # Create a new depth array within the interval with finer sampling
        depths_interval = np.arange(depths[j], depths[j + 1], depth_interval)
        
        depths_fine.extend(depths_interval)
        # Repeat the velocity value within the interval
        vels_fine.extend([vs_mean[j]] * len(depths_interval))

    receptor_result_lst.append([lst_receptor[prof_n-1]]*len(vels_fine))
    depth_result_lst.append(depths_fine)
    velocity_result_lst.append(vels_fine)
        
    # ----------------
    # Plotting results
    
    ax.step(np.insert(vs_mean,[0,-1],[vs_mean[0],vs_mean[-1]]),np.insert(np.cumsum(thick_mean),[0,-1],[0,2])*-1, color='k',lw=2,ls='-',where='post')
    ax.step(np.insert(np.array(vs_mean)+np.array(vs_std),[0,-1],[vs_mean[0]+vs_std[0],vs_mean[-1]+vs_std[-1]]),np.insert(np.cumsum(thick_mean),[0,-1],[0,2])*-1, color='k',lw=1,ls='--',where='post')
    ax.step(np.insert(np.array(vs_mean)-np.array(vs_std),[0,-1],[vs_mean[0]-vs_std[0],vs_mean[-1]-vs_std[-1]]),np.insert(np.cumsum(thick_mean),[0,-1],[0,2])*-1, color='k',lw=1,ls='--',where='post')
    
    ax.yaxis.set_minor_locator(ticker.MultipleLocator(0.1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(0.4))
    
    ax.xaxis.set_minor_locator(ticker.MultipleLocator(100))
    ax.xaxis.set_major_locator(ticker.MultipleLocator(500))

    
    ax.set_xlabel('S-velocity (m/s)')
    ax.set_ylim(-2,0)
    ax.set_xlim(0,1100)
    ax.grid(True, which='both',color='gray', linestyle=':', linewidth=0.25)
    ax.tick_params(axis='both', which='both', direction='in', 
               top=True, bottom=True, left=True, right=True,
               labelbottom=True, labeltop=False, labelleft=False,
               labelright=False)
    ax.set_title('Station '+str(prof_n))
    if prof_n == df_inversion['profile'].unique()[0]:
        ax.set_ylabel('Depth (m)')
        ax.tick_params(axis='both', which='both', direction='in', 
               top=True, bottom=True, left=True, right=True,
               labelbottom=True, labeltop=False, labelleft=True,
               labelright=False)
    if prof_n == df_inversion['profile'].unique()[-1]:
        ax.set_ylabel('Depth (m)')
        ax.yaxis.set_label_position('right')  # Move o rótulo para a direita
        ax.tick_params(axis='both', which='both', direction='in', 
               top=True, bottom=True, left=True, right=True,
               labelbottom=True, labeltop=False, labelleft=False,
               labelright=True)

fig.savefig(figures_path+'observed_predicted_profiles.png')


receptor_result_lst = np.array([item for sublist in receptor_result_lst for item in sublist])
depth_result_lst = np.array([item for sublist in depth_result_lst for item in sublist])
velocity_result_lst = np.array([item for sublist in velocity_result_lst for item in sublist])
   

In [ ]:
# GRID

# ----------
# Input data

coordinates = (receptor_result_lst, depth_result_lst)
velocitys = velocity_result_lst
region = (0,100,-2,0)
    
shape = (100,100)
    
# Now we can set up a gridder for the decimated data
grd_predic = vd.KNeighbors().fit(coordinates, velocitys)
    
# The 'grid' method can still make a geographic grid if we pass in a projection
predicted_grid = grd_predic.grid(region=region,shape=shape,dims=["depth","receptor"],data_names="velocity")

# plotting 
fig,ax = plt.subplots(1, 1, figsize=(10, 5))

im = plt.imshow(predicted_grid.velocity.data, cmap='cividis',extent=[0, 100, -2.0, 0],vmin=100,vmax=1000,aspect=25,origin='lower')

for ix in lst_receptor:
    ax.vlines(x=ix,ymin=-2.0,ymax=0,colors='k',lw=1.0,ls='--',alpha=0.7)
    ax.scatter(x=ix,y=0.05,color='k',marker='v')

ax.hlines(y=0,xmin=0,xmax=100,colors='k',lw=2,ls='-',alpha=1)
ax.set_xlabel('Distance (m)')
ax.grid(which='major', color='gray', linestyle='-', linewidth=0.5)
ax.grid(which='minor', color='gray', linestyle=':', linewidth=0.5, alpha=0.5)
ax.set_ylabel('Depth (m)')
plt.colorbar(im, ax=ax,fraction=0.15, shrink=0.5,label='Shear-wave velocity (m/s)')
fig.savefig(figures_path+'inversion_slice_and_receptors.png')

# Error

In [ ]:
deviation_grid = observed_grid - predicted_grid

In [ ]:
fig,ax = plt.subplots(1, 1, figsize=(10, 5))

im = plt.imshow(deviation_grid.velocity.data, cmap='coolwarm_r',extent=[0, 100, -2.0, 0],vmin=-300,vmax=300,aspect=25,origin='lower')

for idx,ix in enumerate(lst_receptor):
    ax.vlines(x=ix,ymin=-2.0,ymax=0,colors='k',lw=1.0,ls='--',alpha=0.7)
    ax.scatter(x=ix,y=0.05,color='k',marker='v')

ax.hlines(y=0,xmin=0,xmax=100,colors='k',lw=2,ls='-',alpha=1)
ax.set_xlabel('Distance (m)')
ax.grid(which='major', color='gray', linestyle='-', linewidth=0.5)
ax.grid(which='minor', color='gray', linestyle=':', linewidth=0.5, alpha=0.5)
ax.set_ylabel('Depth (m)')
plt.colorbar(im, ax=ax,fraction=0.15, shrink=0.5,label='Deviation (m/s)')
fig.savefig(figures_path+'deviation_observed_predicted_receptors.png')